# Prisoners Dilemma Three Players

In [ ]:
# !pip install gym[classic_control]
# !pip install keras-rl2
# !pip install protobuf==3.20.*

In [1]:
import tensorflow as tf
from tensorflow import keras as k
import numpy as np
import gym
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

from keras import __version__
tf.keras.__version__ = __version__

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory


from gym import Env
from gym.spaces import Discrete, Box, MultiBinary
import numpy as np
import random

from collections import deque

# Setting Up the Environment

In [2]:
def tit_for_tat_all(round,opp1_move,opp2_move):
  if round == 200:
    return 1
  else:
    return opp1_move and opp2_move

def tit_for_tat_any(round,opp1_move,opp2_move):
  if round == 200:
    return 1
  else:
    return opp1_move or opp2_move

In [3]:
def cooperate():
  return 1

def defect():
  return 0

def rand():
  return random.randint(0,1)

In [4]:
def tit_for_two_tat_any(round, opp1_move1, opp1_move2, opp2_move1, opp2_move2):
  if round >= 199 or opp1_move1 == 1 or opp1_move1 == 1 or opp2_move1 or opp2_move2:
    return 1
  else:
    return 0

def tit_for_two_tat_all(round, opp1_move1, opp1_move2, opp2_move1, opp2_move2):
  if round >= 199 or (opp1_move1 == 1 or opp1_move2 == 1) and (opp2_move1 or opp2_move2):
    return 1
  else:
    return 0

In [5]:
def soft_grudger_any(last_defect1, last_defect2):
  if last_defect1 < 4 or last_defect2 < 4:
    return 0
  else:
    return 1

def soft_grudger_all(last_defect1, last_defect2):
  if last_defect1 < 4 and last_defect2 < 4:
    return 0
  else:
    return 1

In [6]:
class ThreePlayers():
    def __init__(self):
        self.action_space = Discrete(2)                      # 0:defect 1:cooperate
        self.observation_space = np.array([MultiBinary(10), MultiBinary(10)])
        self.round = 200

        self.state = np.array([[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]])

        self.opp1_strategy = random.randint(0,9)
        self.opp1_move1=self.state[0][0]
        self.opp1_move2=self.state[0][1]

        self.opp2_strategy = random.randint(0,9)
        self.opp2_move1=self.state[1][0]
        self.opp2_move2=self.state[1][1]

        self.last_move1=1
        self.last_move2=1

        self.opp1_defected=False
        self.opp2_defected=False
        self.user_defected=False

        self.last_defect= np.Infinity
        self.opp1_last_defect= np.Infinity
        self.opp2_last_defect= np.Infinity
        self.move=np.array([random.randint(0,1),random.randint(0,1)])

        self.opp1_score = 0
        self.opp2_score = 0


    def step(self, action):
        self.opp1_move1=self.state[0][0]
        self.opp1_move2=self.state[0][1]

        self.opp2_move1=self.state[1][0]
        self.opp2_move2=self.state[1][1]

        self.last_move2=self.last_move1
        self.last_move1=self.move

        self.move=action


        if self.opp1_strategy == 0:
          opp1_move=tit_for_tat_any(self.round,self.last_move1, self.opp2_move1)
        elif self.opp1_strategy == 1:
          opp1_move=tit_for_tat_all(self.round,self.last_move1, self.opp2_move1)
        elif self.opp1_strategy == 2:
          opp1_move=tit_for_two_tat_all(self.round, self.last_move1, self.last_move2, self.opp2_move1, self.opp2_move2)
        elif self.opp1_strategy == 3:
          opp1_move=tit_for_two_tat_any(self.round, self.last_move1, self.last_move2, self.opp2_move1, self.opp2_move2)
        elif self.opp1_strategy == 4:
          opp1_move=soft_grudger_any(self.last_defect, self.opp2_last_defect)
        elif self.opp1_strategy == 5:
          opp1_move=soft_grudger_all(self.last_defect, self.opp2_last_defect)

        if self.opp2_strategy == 0:
          opp2_move=tit_for_tat_any(self.round,self.last_move1, self.opp1_move1)
        elif self.opp2_strategy == 1:
          opp2_move=tit_for_tat_all(self.round,self.last_move1, self.opp1_move1)
        elif self.opp2_strategy == 2:
          opp2_move=tit_for_two_tat_all(self.round, self.last_move1, self.last_move2, self.opp1_move1, self.opp1_move2)
        elif self.opp2_strategy == 3:
          opp2_move=tit_for_two_tat_any(self.round, self.last_move1, self.last_move2, self.opp1_move1, self.opp1_move2)
        elif self.opp2_strategy == 4:
          opp2_move=soft_grudger_any(self.last_defect, self.opp1_last_defect)
        elif self.opp2_strategy == 5:
          opp2_move=soft_grudger_all(self.last_defect, self.opp1_last_defect)



        temp = np.array(list(self.state[0]))
        for i in range(1,10):
          self.state[0][i] = temp[i-1]
        self.state[0][0] = opp1_move

        temp = np.array(list(self.state[1]))
        for i in range(1,10):
          self.state[1][i] = temp[i-1]
        self.state[1][0] = opp2_move



        if opp1_move == 0:
          self.opp1_defected=True
          self.opp1_last_defect=0
        else:
          self.opp1_last_defect += 1

        if opp2_move == 0:
          self.opp2_defected=True
          self.opp2_last_defect=0
        else:
          self.opp2_last_defect += 1

        if self.move == 0:
          self.user_defected=True
          self.last_defect=0
        else:
          self.last_defect += 1



        if self.move == 1 and opp1_move == 1 and opp2_move ==  1:
          reward = 4
          self.opp1_score += 4
          self.opp2_score += 4
        elif self.move == 1 and opp1_move == 1 and opp2_move == 0:
          reward = 0
          self.opp1_score += 0
          self.opp2_score += 10
        elif self.move == 1 and opp1_move == 0 and opp2_move == 1:
          reward = 0
          self.opp1_score += 10
          self.opp2_score += 0
        elif self.move == 1 and opp1_move == 0 and opp2_move == 0:
          reward = 0
          self.opp1_score += 5
          self.opp2_score += 5
        elif self.move == 0 and opp1_move == 1 and opp2_move ==  1:
          reward = 10
          self.opp1_score += 0
          self.opp2_score += 0
        elif self.move == 0 and opp1_move == 1 and opp2_move == 0:
          reward = 5
          self.opp1_score += 0
          self.opp2_score += 5
        elif self.move == 0 and opp1_move == 0 and opp2_move == 1:
          reward = 5
          self.opp1_score += 5
          self.opp2_score += 0
        elif self.move == 0 and opp1_move == 0 and opp2_move == 0:
          reward = 2
          self.opp1_score += 2
          self.opp2_score += 2

        self.round -= 1

        if self.round == 0:
            done = True
        else:
            done = False

        info = {'opp1_score':self.opp1_score, 'opp2_score':self.opp2_score}


        # Return step information
        return self.state, reward, done, info

    def render(self,mode):
        # Implement viz
        pass

    def reset(self):
        self.round = 200
        self.opp1_strategy = random.randint(0,5)
        self.opp2_strategy = random.randint(0,5)
        self.state = np.array([[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]])
        self.opp1_move1=self.state[0][0]
        self.opp1_move2=self.state[0][1]
        self.opp2_move1=self.state[1][0]
        self.opp2_move2=self.state[1][1]
        self.last_move1=1
        self.last_move2=1
        self.opp1_defected=False
        self.opp2_defected=False
        self.user_defected=False
        self.last_defect= np.Infinity
        self.opp1_last_defect= np.Infinity
        self.opp2_last_defect= np.Infinity
        self.gamma=0.9
        self.move=random.randint(0,1)
        self.reward = 0

        self.opp1_score = 0
        self.opp2_score = 0

        return self.state

In [7]:
env = ThreePlayers()

In [8]:
env.state

array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])

# Test on Random Values

In [12]:
env = ThreePlayers()
episodes = 100
wins=0
for episode in range(1, episodes+1):
    state = env.reset()
    print(env.opp1_strategy, env.opp2_strategy)
    done = False
    score = 0
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    if score > info['opp1_score'] and score > info['opp2_score']:
      wins += 1

    print('Episode:{} Score:{} Info:{}'.format(episode, score, info))
print(f"win rate: {wins/100}")

2 5
Episode:1 Score:313 Info:{'opp1_score': 663, 'opp2_score': 698}
4 4
Episode:2 Score:212 Info:{'opp1_score': 697, 'opp2_score': 697}
4 5
Episode:3 Score:231 Info:{'opp1_score': 781, 'opp2_score': 656}
3 2
Episode:4 Score:1160 Info:{'opp1_score': 345, 'opp2_score': 635}
4 5
Episode:5 Score:230 Info:{'opp1_score': 750, 'opp2_score': 670}
0 0
Episode:6 Score:1418 Info:{'opp1_score': 388, 'opp2_score': 388}
0 3
Episode:7 Score:1352 Info:{'opp1_score': 432, 'opp2_score': 432}
3 0
Episode:8 Score:1370 Info:{'opp1_score': 420, 'opp2_score': 420}
4 5
Episode:9 Score:247 Info:{'opp1_score': 692, 'opp2_score': 637}
5 3
Episode:10 Score:1430 Info:{'opp1_score': 380, 'opp2_score': 380}
5 4
Episode:11 Score:236 Info:{'opp1_score': 656, 'opp2_score': 746}
5 3
Episode:12 Score:1394 Info:{'opp1_score': 404, 'opp2_score': 404}
0 3
Episode:13 Score:1406 Info:{'opp1_score': 396, 'opp2_score': 396}
2 5
Episode:14 Score:367 Info:{'opp1_score': 672, 'opp2_score': 707}
4 3
Episode:15 Score:411 Info:{'opp1

# Deep Learning Model

In [10]:
env.state.shape

(2, 10)

In [13]:
env.observation_space.shape

(2,)

In [14]:
states = env.state.shape
actions = env.action_space.n

In [15]:
states

(2, 10)

In [16]:
actions

2

In [17]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(1, 2, 10), input_dim=3))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='softmax'))
    model.add(Flatten())
    return model

In [19]:
# del model

In [20]:
model = build_model(states, actions)

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 2, 128)         1408      
                                                                 
 dense_1 (Dense)             (None, 1, 2, 128)         16512     
                                                                 
 dense_2 (Dense)             (None, 1, 2, 128)         16512     
                                                                 
 dense_3 (Dense)             (None, 1, 2, 1)           129       
                                                                 
 flatten (Flatten)           (None, 2)                 0         
                                                                 
Total params: 34,561
Trainable params: 34,561
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.input_shape

(None, 1, 2, 10)

In [23]:
env.state.shape

(2, 10)

In [24]:
actions

2

In [25]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions)
    return dqn

In [26]:
dqn = build_agent(model, actions)
dqn.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-3),metrics=['mae'])
#dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=25000, visualize=False)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Training for 25000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 30:15 - reward: 10.0000

c:\Users\Mohammad Doleh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


10000/10000 [==============================] - 75s 8ms/step - reward: 3.2453
50 episodes - episode_reward: 649.060 [192.000, 1460.000] - loss: 12.435 - mae: 2.255 - mean_q: 1.000 - opp1_score: 305.429 - opp2_score: 298.851

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 77s 8ms/step - reward: 3.1312
50 episodes - episode_reward: 626.240 [204.000, 1436.000] - loss: 11.202 - mae: 2.123 - mean_q: 1.000 - opp1_score: 302.924 - opp2_score: 302.485

Interval 3 (20000 steps performed)
 5000/10000 [==============>...............] - ETA: 37s - reward: 2.8786done, took 189.981 seconds


In [27]:
scores = dqn.test(env, nb_episodes=100, visualize=False,verbose=1)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 414.000, steps: 200
Episode 2: reward: 411.000, steps: 200
Episode 3: reward: 411.000, steps: 200
Episode 4: reward: 416.000, steps: 200
Episode 5: reward: 2000.000, steps: 200
Episode 6: reward: 2000.000, steps: 200
Episode 7: reward: 2000.000, steps: 200
Episode 8: reward: 411.000, steps: 200
Episode 9: reward: 2000.000, steps: 200
Episode 10: reward: 411.000, steps: 200
Episode 11: reward: 411.000, steps: 200
Episode 12: reward: 419.000, steps: 200
Episode 13: reward: 422.000, steps: 200
Episode 14: reward: 2000.000, steps: 200
Episode 15: reward: 422.000, steps: 200
Episode 16: reward: 2000.000, steps: 200
Episode 17: reward: 414.000, steps: 200
Episode 18: reward: 422.000, steps: 200
Episode 19: reward: 411.000, steps: 200
Episode 20: reward: 411.000, steps: 200
Episode 21: reward: 416.000, steps: 200
Episode 22: reward: 2000.000, steps: 200
Episode 23: reward: 411.000, steps: 200
Episode 24: reward: 2000.000, steps: 200
Episode 25: 

In [28]:
import numpy as np

wins=0

for episode in range(100):  # Run 100 episodes
    state = env.reset()  # Reset environment for new episode
    done = False
    total_reward = 0

    while not done:
        action = dqn.forward(state)  # Get action from DQN agent
        next_state, reward, done, info = env.step(action)  # Apply action to environment
        total_reward += reward  # Accumulate reward
        state = next_state  # Move to next state

        if done:
          if total_reward > info['opp1_score'] and total_reward > info['opp2_score']:
              wins +=1


print(f'Win Rate: {wins/100}')

Win Rate: 1.0
